In [1]:
import pandas as pd
from pandas._libs.internals import defaultdict
from unidecode import unidecode
from collections.abc import Callable
import re
from typing import Optional
import uuid
import random
from numbers import Number

<h2>Section 1: Reading in the Data</h2>

<h3>Load a CSV into Pandas</h3>

In [2]:
# load a dataframe
infile_path = "../data/shelby_county_properties_municipality_breakdown - s1_2019.csv"
df = pd.read_csv(infile_path)
df

,Board,Number of Contracts,Amount Billed Pilot (INL),Amount Billed Debt Service,Assessment,Tax Based on Assessment at 2021 Tax Rate,Difference,Percentage Total PILOT Billing
0,Downtown Memphis Commission,123,"$1,981,904.47","$1,226,006.18","$349,460,540","$14,153,151.87","-10,945,241.22",20.69%
1,EDGE - Shelby,243,"$2,856,659.78","$6,149,156.86","$673,220,031","$27,265,411.26","-18,259,594.62",58.08%
2,Industrial Development of Arlington,15,"$52,116.90","$114,566.56","$11,977,950","$485,106.98","-318,423.52",1.07%
3,Industrial Development of Bartlett,11,"$15,257.82","$146,630.61","$14,962,783","$605,992.71","-444,104.28",1.04%
4,Industrial Development of Collierville,14,"$774,048.02","$661,885.66","$70,132,723","$2,840,375.28","-1,404,441.60",9.26%
5,Industrial Development of Germantown,4,"$237,740.75","$96,081.97","$15,357,250","$621,968.63","-288,145.91",2.15%
6,Industrial Development of Millington,1,"$10,280.12","$79,383.24","$7,840,320","$317,532.96","-227,869.60",0.58%
7,Health & Education Board - Memphis,73,"$930,606.59",$0.00,"$128,138,985","$5,189,628.89","-4,259,022.30",6.00%
8,Health & Education Board - Shelby,8,"$133,533.27",$0.00,"$27,687,400","$1,121,339.70","-987,806.43",0.86%
9,Miscellaneous Contracts,10,"$35,460.00",$0.00,"$9,033,520","$365,857.56","-330,397.56",0.23%


<h1>Section 2: Cleaning the Columns</h1>

<h3>Clean the column headers</h3>
Often times, column headers have spaces that you cannot see when viewed as a DataFrame. They also often have other qualities like capital letters and special characters which make selecting columns tedious or confusing. Check the column headers and decide what cleaning operations are needed. We can see in the cell below that these headers need:
<ol>
<li>Removal of leading and trailing spaces</li>
<li>Lower case letters only</li>
<li>Removal of special charqacters like "(" </li>
<li>Replacement of between-word spaces with underscores "_"</li>
</ol>

In [3]:
# Examine the strings that make the column headers
df.columns

Index(['    Board    ', '    Number of Contracts    ',
       '    Amount Billed Pilot (INL)    ', '    Amount Billed Debt Service',
       'Assessment    ', '    Tax Based on Assessment at 2021 Tax Rate',
       'Difference    ', '    Percentage Total PILOT Billing'],
      dtype='object')

<h2>Define Cleaning Functions for Cleaning Individual Entries</h2>
First, we will define a number of functions for cleaning an entry of any data type which we can later apply where needed.

In [4]:
# Remove special characters when dealing with words
# transforms removed characters into spaces, removes . and -
def remove_special_for_words(entry: str | Number):
    return re.sub(r"[^a-zA-Z0-9]+", ' ', str(entry))

# Remove leading and trailing spaces
def truncate(entry: str | Number):
    return str(entry).strip()

# Replace spaces with underscores
def snake_case(entry: str | Number):
    return str(entry).replace(" ", "_")

# Make all letters lowercase
def lower_case(entry: str | Number):
    return str(entry).lower()

<h2>Define a Function to Apply Any Number of Sub-functions (above) to All Column Headers</h2>
By having a separate function which applies any number of smaller functions to a columns headers, we can clean the headers more quickly and easily than we would if we tried to do everything at once.

In [5]:
# takes in a list of functions and applies those functions to all headers of a pandas dataframe
def clean_headers(df: pd.DataFrame, cleaning_functions_list:list[Callable]) -> pd.DataFrame:
    column_name_map = {item: item for item in df.columns}
    for item in column_name_map:
        for function in cleaning_functions_list:
            column_name_map[item] = function(column_name_map[item])
    df.rename(columns=column_name_map, inplace=True)
    return df

<h2>Clean the Headers</h2>
Apply the string cleaning functions we want in the correct order to the column headers.

In [6]:
# Apply functions to the columns headers to remove special characters, remove leading and trailing spaces, put headers in snake case, and make all letters lower case
df = clean_headers(df, [remove_special_for_words, truncate, snake_case, lower_case])
# Check the new column names
print(df.columns)
# View the new dataframe
df


Index(['board', 'number_of_contracts', 'amount_billed_pilot_inl',
       'amount_billed_debt_service', 'assessment',
       'tax_based_on_assessment_at_2021_tax_rate', 'difference',
       'percentage_total_pilot_billing'],
      dtype='object')


,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
0,Downtown Memphis Commission,123,"$1,981,904.47","$1,226,006.18","$349,460,540","$14,153,151.87","-10,945,241.22",20.69%
1,EDGE - Shelby,243,"$2,856,659.78","$6,149,156.86","$673,220,031","$27,265,411.26","-18,259,594.62",58.08%
2,Industrial Development of Arlington,15,"$52,116.90","$114,566.56","$11,977,950","$485,106.98","-318,423.52",1.07%
3,Industrial Development of Bartlett,11,"$15,257.82","$146,630.61","$14,962,783","$605,992.71","-444,104.28",1.04%
4,Industrial Development of Collierville,14,"$774,048.02","$661,885.66","$70,132,723","$2,840,375.28","-1,404,441.60",9.26%
5,Industrial Development of Germantown,4,"$237,740.75","$96,081.97","$15,357,250","$621,968.63","-288,145.91",2.15%
6,Industrial Development of Millington,1,"$10,280.12","$79,383.24","$7,840,320","$317,532.96","-227,869.60",0.58%
7,Health & Education Board - Memphis,73,"$930,606.59",$0.00,"$128,138,985","$5,189,628.89","-4,259,022.30",6.00%
8,Health & Education Board - Shelby,8,"$133,533.27",$0.00,"$27,687,400","$1,121,339.70","-987,806.43",0.86%
9,Miscellaneous Contracts,10,"$35,460.00",$0.00,"$9,033,520","$365,857.56","-330,397.56",0.23%


<h1>Section 3: Cleaning the Columns</h3>

<h2>Define a Function for Applying Any Number of Sub-Functions to Selected Columns of a DataFrame</h2>
<p>Just like before, we want to be able to select the cleaning functions we want and apply them all at once in the correct order. This means that we want a function that takes in our entry-cleaning functions as arguments and applys them to an entire column of a DataFrame</p>

In [7]:
# Apply a list of functions to a list of columns
def clean_columns(df: pd.DataFrame, 
                  selected_columns: list[str], 
                  cleaning_functions_list:list[Callable]) -> pd.DataFrame:
    for col in selected_columns:
        for function in cleaning_functions_list:
            df[col] = df[col].apply(lambda x: function(x))
    return df

<h2>Clean the String Columns</h2>
<p>Columns containing string-like data often need to be cleaned using different functions than the column headers. In this example, the "board" column contains a number of names of different development boards in Shelbey, TN. We want to make these all lower case and to remove leading and trailing spaces, but we do not want to put it in snake case. Doing this makes it easier to compare this data to other dataframes featuring the same board names later.</p>
<p></p>


In [8]:
# Apply the correct entry-cleaning functions to each of our string columns. Note that we only have one in this example, but we could add as many as we like to the "string_columns" list if we want more

string_columns = ["board"]
df = clean_columns(df, string_columns, [remove_special_for_words, truncate, lower_case])
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
0,downtown memphis commission,123,"$1,981,904.47","$1,226,006.18","$349,460,540","$14,153,151.87","-10,945,241.22",20.69%
1,edge shelby,243,"$2,856,659.78","$6,149,156.86","$673,220,031","$27,265,411.26","-18,259,594.62",58.08%
2,industrial development of arlington,15,"$52,116.90","$114,566.56","$11,977,950","$485,106.98","-318,423.52",1.07%
3,industrial development of bartlett,11,"$15,257.82","$146,630.61","$14,962,783","$605,992.71","-444,104.28",1.04%
4,industrial development of collierville,14,"$774,048.02","$661,885.66","$70,132,723","$2,840,375.28","-1,404,441.60",9.26%
5,industrial development of germantown,4,"$237,740.75","$96,081.97","$15,357,250","$621,968.63","-288,145.91",2.15%
6,industrial development of millington,1,"$10,280.12","$79,383.24","$7,840,320","$317,532.96","-227,869.60",0.58%
7,health education board memphis,73,"$930,606.59",$0.00,"$128,138,985","$5,189,628.89","-4,259,022.30",6.00%
8,health education board shelby,8,"$133,533.27",$0.00,"$27,687,400","$1,121,339.70","-987,806.43",0.86%
9,miscellaneous contracts,10,"$35,460.00",$0.00,"$9,033,520","$365,857.56","-330,397.56",0.23%


<h2>Clean the Numeric Columns</h2>
<p>The numeric columns in this dataframe contain some special characters which we want to keep (periods and negative signs) and some which we must remove (dollar signs and commas). We need a new function to remove only the special characters we want to remove.</p>

<h4>Define a Function to Remove Only Non-Number-Related Special Characters</h4>

In [9]:
# Remove special characters when dealing with numbers
# No spaces, does not remove - and .
def remove_special_for_numbers(my_string: str | Number):
    return re.sub(r"[^a-zA-Z0-9-.]+", '', str(my_string))

<h4>Apply the Function to the Numeric Columns</h4>

In [10]:
# Clean the numeric columns
numeric_columns = ['amount_billed_pilot_inl', 
                   'amount_billed_debt_service', 
                   'assessment',
                   'tax_based_on_assessment_at_2021_tax_rate', 
                   'difference',
                   'percentage_total_pilot_billing']
df = clean_columns(df, numeric_columns, [remove_special_for_numbers])
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
0,downtown memphis commission,123,1981904.47,1226006.18,349460540,14153151.87,-10945241.22,20.69
1,edge shelby,243,2856659.78,6149156.86,673220031,27265411.26,-18259594.62,58.08
2,industrial development of arlington,15,52116.90,114566.56,11977950,485106.98,-318423.52,1.07
3,industrial development of bartlett,11,15257.82,146630.61,14962783,605992.71,-444104.28,1.04
4,industrial development of collierville,14,774048.02,661885.66,70132723,2840375.28,-1404441.60,9.26
5,industrial development of germantown,4,237740.75,96081.97,15357250,621968.63,-288145.91,2.15
6,industrial development of millington,1,10280.12,79383.24,7840320,317532.96,-227869.60,0.58
7,health education board memphis,73,930606.59,0.00,128138985,5189628.89,-4259022.30,6.00
8,health education board shelby,8,133533.27,0.00,27687400,1121339.70,-987806.43,0.86
9,miscellaneous contracts,10,35460.00,0.00,9033520,365857.56,-330397.56,0.23


<h2>Section 4: Creating a Unique ID Column</h2>

<h3>Defining a Deterministic Hash Function (Advanced)</h3>
<p>Sometimes, data does not come with a unique identifier string column. Pandas automatically creates an index column which ennumerates our data (0, 1, 2, 3 ...) but this is not the recommended way to identify unique observations (entries) in a dataframe. When we perform data joins, we want our identifying column to always contain the same unique value when associated with the same object, and for our identifiers to never overlap. Using consecutive numbers prevents overlap, but it can create confusion. Is this really the identifying column, or was it just generated by default ages ago, and everyone forgot about it? In a small dataset where few joins are performed it may not matter, but in a large one it definitely does. </p>

<p>Lets create our own unique identifier based on the values in the "board" column. We want to be sure that no matter when we run this function, we will always get the same unique identification string when we input the same "board" value. This way, we can create anonymous unique identifiers using different datasets with different boards, and if any boards happen to be the same, they will still be associated with the same identifier.</p>

In [11]:
# Create a unique alphanumeric string by taking a deterministic hash of an input value
def deterministic_uuid(data:str):
    random.seed(data)
    id = uuid.UUID(bytes=bytes(random.getrandbits(8) for _ in range(16)), version=4)
    id = str(id).replace("-", "")
    return id

# Apply the function above to a DataFrame, using any number of selected columns as identifiers
# Doing it this way allows us to use more than one column to identify unique data points (like if we wanted to use a persons first name AND last name, each in different columns, for instance).
def set_unique_index(df: pd.DataFrame, columns_to_hash: list[str], index_name="id", index_length_limit: Optional[int] = None):
    df[index_name] = list(map(lambda x: deterministic_uuid(''.join([str(col_value) for col_value in x]))[0:index_length_limit], df[columns_to_hash].values))
    df.set_index(index_name, inplace=True)
    return df

<h3>Apply the Functions to Create The ID Column</h3>
<p> Now we will simply apply the functions we defined above, using a list of uniquely-identifying columns (in this case, just "board") to distinguish one observation from another.

In [12]:
# Create a unique index column based on the values in the identifying column
# Note: Ids are based on a deterministic hash, which means they are 
# reproducible even if the function is run by a different user at a different time,
# as long as the string in the identifying column is the same. 
identifying_columns = ["board"]

# The argument "index length limit" shortens the ID value generated to something more reasonable for this size dataset. By default, the id string is much longer.
df = set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12)
df

,board,number_of_contracts,amount_billed_pilot_inl,amount_billed_debt_service,assessment,tax_based_on_assessment_at_2021_tax_rate,difference,percentage_total_pilot_billing
board_id,,,,,,,,
5b1f4c0c65a3,downtown memphis commission,123,1981904.47,1226006.18,349460540,14153151.87,-10945241.22,20.69
d5fe3a85ec4e,edge shelby,243,2856659.78,6149156.86,673220031,27265411.26,-18259594.62,58.08
abb34861ca91,industrial development of arlington,15,52116.90,114566.56,11977950,485106.98,-318423.52,1.07
aada4183f85c,industrial development of bartlett,11,15257.82,146630.61,14962783,605992.71,-444104.28,1.04
91e543f3085f,industrial development of collierville,14,774048.02,661885.66,70132723,2840375.28,-1404441.60,9.26
3f1cf0abc31c,industrial development of germantown,4,237740.75,96081.97,15357250,621968.63,-288145.91,2.15
757a7152194c,industrial development of millington,1,10280.12,79383.24,7840320,317532.96,-227869.60,0.58
3bcfcbd3a12d,health education board memphis,73,930606.59,0.00,128138985,5189628.89,-4259022.30,6.00
64c9e33b120e,health education board shelby,8,133533.27,0.00,27687400,1121339.70,-987806.43,0.86


<h1>Section 5: Casting Data Types</h1>
<p>Pandas variables are stored with data types internal to the pandas library. By default, if pandas is uncertain what the data type of a column is, that column is stored as an "object" data type. This type is the hardest to manipulate. We want to avoid storing data as objects unless we have no choice. Casting our data forces us to create handy schema that we can use for reading the data back in later immediately as the correct data type without having to do these operations again. 
    

<h3>Check the Current Data Types</h3>

In [13]:
# Check the current data types of the columns
df.dtypes

board                                       object
number_of_contracts                          int64
amount_billed_pilot_inl                     object
amount_billed_debt_service                  object
assessment                                  object
tax_based_on_assessment_at_2021_tax_rate    object
difference                                  object
percentage_total_pilot_billing              object
dtype: object

<h3>Define a Function for Casting Data Types</h3>

In [14]:
# Define a function to cast the data types of all DataFrame Variables using a dictionary of variable names to data types, as strings.
def cast_data_types(df: pd.DataFrame, names_to_types: dict[str,str]) -> pd.DataFrame:
    return df.astype(names_to_types)



<h3>Apply the Function and Cast the Data</h3>

In [15]:
# Apply our function to our data using a dictionary column names to desired data types
names_to_types = {"board": "string", 
                  "number_of_contracts": "int64", 
                  "amount_billed_pilot_inl": "float64", 
                  "amount_billed_debt_service": "float64", 
                  "assessment": "float64",
                  "tax_based_on_assessment_at_2021_tax_rate": "float64", 
                  "difference": "float64",
                  "percentage_total_pilot_billing": "float64",
                  }
df = cast_data_types(df, names_to_types)

# Check the new data types of the columns
df.dtypes

board                                       string[python]
number_of_contracts                                  int64
amount_billed_pilot_inl                            float64
amount_billed_debt_service                         float64
assessment                                         float64
tax_based_on_assessment_at_2021_tax_rate           float64
difference                                         float64
percentage_total_pilot_billing                     float64
dtype: object

<h1>Section 6: Writing Out the Clean Data</h1>
<p>Now that the data is clean, we want to write out a clean copy of the data to a new destination. We also may optionally want to convert the outfile to utf-8 format, as this format is the most versitile and can be read by the most sources.

<h3>Write Out the Data</h3>

In [16]:
# Save the now-clean csv
outfile_path = "../data/processed/s1_2019.csv"
df.to_csv(outfile_path, sep=",")

<h3>Define a Function to Convert a File to UTF-8 Format</h3>

In [17]:
# convert a csv into utf8 format
# Note that this function will replace any remaining non-utf-8-compatible special characters in the data with its nearest utf-8 equivalent.

def convert_utf8(original_file_path: str, new_file_path: str):
    df = pd.read_csv(original_file_path, converters=defaultdict(lambda i: str))
    for column in df.columns:
        df[column] = df[column].apply(lambda x: unidecode(str(x)))
    df.to_csv(new_file_path, encoding='utf-8')

<h3>Convert the Outfile to UTF8 Format</h3>

In [18]:
convert_utf8(original_file_path=outfile_path, new_file_path=outfile_path)

<h1>Section 7: Next Steps</h1>
<p>Now that you have the basic tools to clean a dataset, you'll need some functions to manipulate that data easily for presenting summary statistics and charts. In a future notebook, I will present functions for gaining a quick snapshot of a dataset.